In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [26]:
# print(openai_api_key)

In [27]:
ref_form = r"""方程组：
                解：$\left\{\begin{array}{l}5 x+4 y=6,① \\ 2 x+3 y=1 ;②\end{array}\right.$
由$①×2$，得：$10x + 8y = 12.③$ 
由$②×5$，得： $10x + 15y = 5.④$
$③-④$，得： $-7y = 7,$
解得：$y=-1$.
把$y=-1$代入$①$，得：$5x-4=6$,
解得：$x=2$.
"""

ref = """
    让我们一步一步来看，以得到正确答案
"""

question = """
    在空间直角坐标系中，设单叶双曲面 $\Gamma$ 的方程为 $x^2+y^2-z^2=1$,设 $P$ 为空间的平面, 它交 $\Gamma$ 于一抛物线 $C$. 求该平面的法线与 $z$- 轴的夹角.
"""

template_note = """
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。

    Goals:
    根据给定信息按照要求像老师一样讲解一个题目，但不要输出任何讲解内容，只输出知识点和板书。

    技能:
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目：
    1. 给出该题目涉及的知识点
    2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤

    原题和参考信息:
    题目:{question}
    参考信息:{ref}

    严格按照以下列字符串格式输出:
    知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ###板书一:    ###板书二:

    注意事项:
    1. 你擅长利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
    2. 你的解题思路、方法、解答需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
    3. 知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。
    4. 板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书单元。
    5. 每个式子要独立成行，不输出连续很长的一个式子。
    6. 注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号。
    7. 禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
    8. 如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
    9. 板书和知识点讲解之外，不能出现任何讲解内容。
    10. 返回的字段中只能出现知识点讲解、板书N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应，不允许出现其他字段。在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
    11. 板书参考书写范式，你需要根据题型参照范式输出板书：
        {ref_form}
    12. 其它注意事项：
        a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
        b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
        c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。

    注意：严格遵守上述规则，不输出任何讲解内容。
    下面是对话的开头：
    学生: 老师请帮我看看这题怎么做？
    老师：这道题目的考查要点是[], 解题关键点是[], 让我们一步一步来思考, 以确保我们有正确的答案：

"""
    
template_expl = """
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ###讲解一:    ###讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 请确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。

    下面是板书的内容：
    {notes}
"""


In [29]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

llm_note = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=1000,
    timeout=None,
    max_retries=2,
)

llm_expl = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=1000,
    timeout=None,
    max_retries=2,
)



prompt_note = PromptTemplate.from_template(template_note)
prompt_expl = PromptTemplate.from_template(template_expl)

stream_note = ''
stream_expl = ''

parser = StrOutputParser()
chain_note = prompt_note | llm_note | parser
chain_expl = prompt_expl | llm_expl | parser
counter = 0

async def test(s: str):
    global stream_expl
    async for chunk in chain_expl.astream({"question": question, "ref": ref, "notes": stream_note}, version="v2"):
        print(chunk, end='', flush=True)
        stream_expl += chunk

async for chunk in chain_note.astream({"question": question, "ref_form": ref_form, "ref": ref}, version="v2"):
    # global stream_now
    print(chunk, end='', flush=True)
    stream_note += chunk
    

知识点讲解:本题考查的知识点是空间直角坐标系中的曲面方程与平面方程的交线问题，解题关键点是求平面的法线与坐标轴的夹角。

###板书一:
设平面 $P$ 的方程为 $Ax + By + Cz = D$

平面 $P$ 交单叶双曲面 $\Gamma$ 于抛物线 $C$

由题意得
$x^2 + y^2 - z^2 = 1$
$Ax + By + Cz = D$

设 $z = kx + ly + m$

代入 $\Gamma$ 的方程得
$x^2 + y^2 - (kx + ly + m)^2 = 1$

展开并整理得
$x^2 + y^2 - (k^2x^2 + 2klxy + l^2y^2 + 2kmx + 2lmy + m^2) = 1$

$x^2(1 - k^2) + y^2(1 - l^2) - 2klxy - 2kmx - 2lmy - m^2 = 1$

###板书二:
平面 $P$ 的法向量为 $(A, B, C)$

设法向量与 $z$ 轴的夹角为 $\theta$

由题意得
$\cos \theta = \frac{C}{\sqrt{A^2 + B^2 + C^2}}$

由抛物线方程得
$A = 0, B = 0, C = 1$

故
$\cos \theta = \frac{1}{\sqrt{0^2 + 0^2 + 1^2}} = 1$

$\theta = \arccos(1) = 0$

故答案是 $0$

In [30]:
await test(stream_note)

###讲解一:
首先，我们设定平面 $P$ 的方程为 $Ax + By + Cz = D$。这个方程表示一个平面在空间中的位置，其中 $A, B, C$ 是平面的法向量的分量，$D$ 是常数。

接下来，题目告诉我们平面 $P$ 交单叶双曲面 $\Gamma$ 于一条抛物线 $C$。单叶双曲面的方程是 $x^2 + y^2 - z^2 = 1$。为了找到交线，我们需要将平面方程和双曲面方程结合起来。

我们假设 $z$ 可以表示为 $z = kx + ly + m$，其中 $k, l, m$ 是待定系数。将这个假设代入双曲面方程 $x^2 + y^2 - z^2 = 1$，我们得到：

$x^2 + y^2 - (kx + ly + m)^2 = 1$

接下来，我们展开并整理这个方程：

$x^2 + y^2 - (k^2x^2 + 2klxy + l^2y^2 + 2kmx + 2lmy + m^2) = 1$

将同类项合并，我们得到：

$x^2(1 - k^2) + y^2(1 - l^2) - 2klxy - 2kmx - 2lmy - m^2 = 1$

这个方程表示了平面 $P$ 和双曲面 $\Gamma$ 的交线的方程形式。

###讲解二:
接下来，我们需要求平面 $P$ 的法向量与 $z$ 轴的夹角。平面 $P$ 的法向量为 $(A, B, C)$。

设法向量与 $z$ 轴的夹角为 $\theta$，根据向量夹角的定义，我们有：

$\cos \theta = \frac{C}{\sqrt{A^2 + B^2 + C^2}}$

题目中提到交线是抛物线，这意味着平面 $P$ 必须平行于 $z$ 轴的某个平面。由抛物线方程的特性，我们可以推断 $A = 0, B = 0, C = 1$。

因此，代入上面的公式，我们得到：

$\cos \theta = \frac{1}{\sqrt{0^2 + 0^2 + 1^2}} = 1$

所以，$\theta = \arccos(1) = 0$。

这意味着平面的法向量与 $z$ 轴的夹角为 $0$ 度，即平面 $P$ 的法向量与 $z$ 轴平行。

故答案是 $0$。

In [31]:
template_orig = """
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一讲解一个题目
    Goals:
    根据给定信息按照要求像老师一样讲解一个题目

    技能
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目
        1. 给出该题目涉及的知识点
        2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤，并对应生成讲解；如果没有原题，则参照相似题的思路和解法给出结果

    第一部分:原题和参考信息
        题目:{question}
        学段：初中，你需要适配初中的知识点，解法和书写表达不超纲
        参考信息:{ref}
    第二部分: 以下列字符串格式输出
        知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ###板书一:     ###讲解一:       ###板书二:       ###讲解二:      ...
    注意事项:
        1.你擅将利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
        2.你的解题思路、方法、讲解需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
        3.知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。

        4.板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书+讲解单元；讲解是对应板书模块的说明。在板书的最开始按需写出"解："或"证明："，在板书最后写出"答："或"故答案是："。
        5.每个式子要独立成行，不输出连续很长的一个式子。
        6.注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号
        7.禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
        8.如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
        9.讲解和板书一一对应，讲解要重点强调对应板书涉及到的思路和方法，讲解要抓住题目重点，言简意赅，语言精炼。  
        例如:
        "知识点讲解:本题考查的知识点是.....，解题关键点是......。###板书一: $\because$ $\frac{{x^2-2*x-1}}{{x-1}}=\frac{{(x-1)^2}}{{x-1}}=(x-1)^{{2-1}}=x-1$###讲解一:首先利用完全平方公式将分子化简,随后发现分子分母都是(x-1)的次幂,因此将指数相减即可。###板书二: $\therefore$ $\frac{{x^2-2*x-1}}{{x-1}}*(x-2)=(x-1)(x-2)=x^2-3*x+2$ \n故答案是$x^2-3*x+2$###讲解二:根据上述化简结果,代入原式即可得到最终答案$x^2-3*x+2$"
        10.返回的字段中只能出现知识点讲解、板书N、讲解N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应...不能出任何现其他的字段，在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
        11.板书参考书写范式，你需要根据题型参照范式输出板书：
            {ref_form}
        12.其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题别用括号1234。
"""

In [33]:
llm_orig = ChatOpenAI(
    model='gpt-4o',
    temperature=0,
    max_tokens=1000,
    timeout=None,
    max_retries=2,
)

prompt_orig = PromptTemplate.from_template(template_orig)

stream_orig = ''

chain_orig = prompt_orig | llm_orig | parser

async for chunk in chain_orig.astream({"question": question, "ref_form": ref_form, "ref": ref}, version="v2"):
    print(chunk, end='', flush=True)
    # stream_orig += chunk

知识点讲解:本题考查的知识点是空间直角坐标系中的曲面方程与平面方程的交线，解题关键点是利用平面方程与双曲面方程联立求解交线方程，并通过法向量计算夹角。

###板书一:
设平面 $P$ 的方程为 $Ax + By + Cz = D$

由题意，平面 $P$ 交单叶双曲面 $\Gamma$ 于一抛物线 $C$

联立方程：
$x^2 + y^2 - z^2 = 1$
$Ax + By + Cz = D$

###讲解一:
首先设平面 $P$ 的方程为 $Ax + By + Cz = D$，然后将平面方程与双曲面方程联立，得到交线的方程。

###板书二:
将 $z$ 用平面方程表示，得：
$z = \frac{D - Ax - By}{C}$

代入双曲面方程，得：
$x^2 + y^2 - \left(\frac{D - Ax - By}{C}\right)^2 = 1$

###讲解二:
将 $z$ 用平面方程表示，并代入双曲面方程，得到关于 $x$ 和 $y$ 的方程。

###板书三:
化简得：
$x^2 + y^2 - \frac{(D - Ax - By)^2}{C^2} = 1$

###讲解三:
将代入后的方程化简，得到一个关于 $x$ 和 $y$ 的方程。

###板书四:
平面 $P$ 的法向量为 $(A, B, C)$

$z$ 轴的方向向量为 $(0, 0, 1)$

法向量与 $z$ 轴的夹角 $\theta$ 满足：
$\cos \theta = \frac{(A, B, C) \cdot (0, 0, 1)}{|(A, B, C)| \cdot |(0, 0, 1)|}$

###讲解四:
平面 $P$ 的法向量为 $(A, B, C)$，$z$ 轴的方向向量为 $(0, 0, 1)$，利用向量点积公式计算夹角。

###板书五:
$\cos \theta = \frac{C}{\sqrt{A^2 + B^2 + C^2}}$

$\theta = \arccos \left(\frac{C}{\sqrt{A^2 + B^2 + C^2}}\right)$

###讲解五:
通过计算得到 $\cos \theta$ 的表达式，并求出夹角 $\theta$。

故答案是：$\theta = \arccos \left(\frac{C}{\s

In [34]:
# importing os module  
import os 
import pprint 
  
# Get the list of user's 
home = os.environ['HOME'] 
  
print("HOME:", home) 


HOME: /Users/eureka
